![example](images/director_shot.jpeg)

# IMDB Project Phase 1

**Authors:** Brett Zimmerman, Sanjit Varma, Douglas Lu, Tony Bai
***

## Overview

Microsoft is attempting to break into original content film making by creating a new movie studio. Using data from online movie resource websites such as Rotten Tomatoes and IMDB, our group cross analyzed Revenue agasint, release month, genre and ----. Through this analysis we are able to recommend 

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [5]:
# Import standard packages #DOUG
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

%matplotlib inline


In [14]:
# Here you run your code to explore the data
dfbom = pd.read_csv('data/zippeddata/bom.movie_gross.csv')

df_imdb_name = pd.read_csv('data/zippeddata/imdb.name.basics.csv')

df_imdb_titleb = pd.read_csv('data/zippeddata/imdb.title.basics.csv')

df_imdb_title = pd.read_csv('data/zippeddata/imdb.title.crew.csv')

df_imdb_principals = pd.read_csv('data/zippeddata/imdb.title.principals.csv')

df_imdb_ratings = pd.read_csv('data/zippeddata/imdb.title.ratings.csv')

tsv_file='data/zippeddata/rt.movie_info.tsv'
csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('rt.movie_info.csv',index=False)
df_rt_movie = pd.read_csv('rt.movie_info.csv')

tsv_file='data/zippeddata/rt.reviews.tsv'
#csv_table=pd.read_table(tsv_file,sep='\t')
csv_table.to_csv('rt.reviews.csv',index=False)
df_rt_review = pd.read_csv('rt.reviews.csv')

df_tmdb_movies = pd.read_csv('data/zippeddata/tmdb.movies.csv')

df_budget = pd.read_csv('data/zippeddata/tn.movie_budgets.csv')

In [15]:
df_list = [dfbom, df_imdb_name, df_imdb_titleb, df_imdb_title, df_rt_review, 
        df_imdb_principals, df_imdb_ratings, df_tmdb_movies, df_budget, df_rt_movie]

for df in df_list:
        display(df.head())

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [16]:
df_imdb_titleb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [10]:
# cleaned the tdmb Budget data set
# converted the last 3 col to float and removed the $

df_budget[df_budget.columns[3:6]] = df_budget[df_budget.columns[3:6]].replace('[\$,]', '', 
                                                                              regex=True).astype(float)

# created another row to determine wether the film grossed a profit
df_budget['profit'] = df_budget['worldwide_gross']- df_budget['production_budget']

# created a row of booleans returning True if profit exceeds 1.5 production budget
df_budget['fin_success'] =  (df_budget['profit'] > df_budget['production_budget']*1.5)




#combined data frame from budget and imdb title to obtain a combined table with genre 
df_budget_genre = df_budget.merge(df_imdb_titleb, 
                                  left_on='movie', 
                                  right_on='primary_title') 

#clean the genres into rows so they can be categorized easier
df_budget_genre['genres'] = df_budget_genre['genres'].str.strip().str.split(',')
single_genre = df_budget_genre.explode('genres')


# created another DF combining the single_genres into the budget
budget_genre= df_budget.merge(single_genre, left_on='movie', right_on='primary_title')

#dropped unnecessary cols
budget_genre.drop(columns=['id_x','movie_x', 'id_y', 'release_date_y', 
                           'production_budget_y', 'domestic_gross_y', 'worldwide_gross_y', 
                           'profit_y', 'fin_success_y', 'tconst','primary_title', 
                           'original_title', 'start_year' ], inplace = True )


#created a col for release month
budget_genre['release_month'] = df_budget_genre['release_date'].map(lambda x : x[:3])

#created a col for release year
budget_genre['release_year'] = budget_genre['release_date_x'].map(lambda x : x[-4:]).astype(int)


#selected films between 2010- 2019
budget_genre = budget_genre.loc[(budget_genre['release_year'] > 2009) & (budget_genre['release_year'] < 2020)]

#dropped duplicates of films
budget_genre.drop_duplicates(subset=['movie_y','genres'],keep='first',inplace=True)


budget_genre.groupby('release_year')['genres'].count()

release_year
2010    515
2011    579
2012    539
2013    575
2014    631
2015    819
2016    548
2017    433
2018    370
2019    148
Name: genres, dtype: int64

In [16]:
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,fin_success
0,1,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09,2.351345e+09,True
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,6.350639e+08,True
2,3,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-2.002376e+08,False
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,1.072414e+09,True
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09,9.997217e+08,True


In [4]:
# analysis of run time vs success rate

budget_genre['runtime_minutes'].describe()

df_film_length = budget_genre[budget_genre['runtime_minutes'] > 60]

df_film_length.describe()


df_film_length['length'] = df_film_length['runtime_minutes'].map(lambda x : 
                                                                 'Short' if x < 90 
                                                                 else('Medium' if x < 150 else 'Long'))


length_total = dict(df_film_length.groupby('length')['genres'].count())
length_success = dict(df_film_length[df_film_length['fin_success_x'] == True].groupby('length')['genres'].count())

g1= length_success
g2= length_total

g3={x:float(g1[x])/g2[x] for x in g2}


fig_dims = (10, 8)
fig, ax = plt.subplots(figsize=fig_dims)
sns.scatterplot(data=df_film_length, x="runtime_minutes", y="profit_x", hue="length")


NameError: name 'budget_genre' is not defined

In [11]:
length_percent = pd.DataFrame.from_dict(g3, orient='index')


ax = length_percent.plot(kind='bar', figsize=(10, 8), fontsize=14, 
                         legend= False, colormap = 'rainbow', ylim=(0,1))

ax.set_title("Movie Length vs. % of Success", fontsize =20)
ax.set_ylabel('Success Rate', fontsize= 18)
ax.set_xlabel("Movie Length", fontsize=18)

for i in ax.patches:
    
    ax.text(i.get_x()+.1, i.get_height()+.03, \
            str(round((i.get_height())*100, 2))+'%', fontsize=14,
                color='black')
    


NameError: name 'g3' is not defined

In [6]:
#cleaning slicing only the month from the data
df_budget_genre['release_month'] = df_budget_genre['release_date'].map(lambda x : x[:3])

# df_budget_genre[df_budget_genre['fin_success'] == True].groupby('release_month')['id'].count()

#compiling the wins per month into a category 

month_success_total = dict(df_budget_genre[df_budget_genre['fin_success'] == True].groupby('release_month')['id'].count())

month_total = dict(df_budget_genre.groupby('release_month')['id'].count())
                   
m1= month_success_total
m2= month_total

m3={x:float(m1[x])/m2[x] for x in m2}

month_percent = pd.DataFrame.from_dict(m3, orient= 'index')

#rename the col
# .sort_values(by='id', ascending = False)

month_percent.rename(columns = {month_percent.columns[0] : "success rate %"}, inplace=True)

x= month_percent.sort_values(by = "success rate %", ascending = False)

ax = x.plot(kind ='bar', fontsize= 18, colormap = 'autumn', figsize =(14, 10), ylim = (0, 1) )

ax.set_title("Release Month Vs Financial Success", fontsize = 20)
ax.set_ylabel('Success Rate', fontsize= 18)
ax.set_xlabel("Release Month", fontsize=18)

#annotating the bar graph 

for i in ax.patches:
    
    ax.text(i.get_x(), i.get_height()+.03, \
            str(round((i.get_height())*100, 2))+'%', 
            fontsize=14, color='black')
    
#release month totals table

df_month_release_totals = pd.DataFrame.from_dict(m2, orient= 'index')
#rename col to count
df_month_release_totals.rename(columns = {df_month_release_totals.columns[0] : "# of Releases"}, inplace=True)


NameError: name 'df_budget_genre' is not defined

In [7]:
budget_genre.groupby('genres')['genres'].count().sort_values(ascending= False) # sorted genres by data

genre_total= dict(budget_genre.groupby('genres')['genres'].count().sort_values(ascending= False))

# budget_genre[budget_genre['fin_success'] == True].groupby('genres').count().sort_values('fin_success', ascending= False)

genre_success_total= dict(budget_genre[budget_genre['fin_success_x'] == True].groupby('genres')['genres'].count().sort_values(ascending = False))

#delete news and reality tv form data set

del genre_total['News']
del genre_total['Reality-TV']

len(genre_total)

genre_success_total

#delete News from genre_success_total so that both lists match

del genre_success_total['News']
len(genre_total)== len(genre_success_total)

NameError: name 'budget_genre' is not defined

In [8]:
budget_genre[budget_genre['fin_success_x'] == True].groupby('genres')['genres'].count().sum()

NameError: name 'budget_genre' is not defined

In [9]:
budget_genre.groupby('genres')['genres'].count().sum()

NameError: name 'budget_genre' is not defined

In [13]:
# return the percentages of success of each genre

d1= genre_success_total
d2= genre_total

d3={x:float(d1[x])/d2[x] for x in d2}

d3={k: v for k, v in sorted(d3.items(), key=lambda item: item[1])}


NameError: name 'genre_success_total' is not defined

In [12]:
df_percentage = pd.DataFrame.from_dict(d3, orient= 'index')

ax = df_percentage.plot.barh(figsize =(10, 8), legend=False)
ax.set_title("Genres vs. Success", fontsize=20)
ax.set_ylabel('Genres', fontsize= 16)
ax.set_xlabel("% of Success", fontsize=16)
ax.set_xlim(0,1)

for i in ax.patches:
    
    ax.text(i.get_width(), i.get_y()+.18, \
            str(round((i.get_width())*100, 2)) +'%', fontsize=13, color='Black')

NameError: name 'd3' is not defined

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***